In [1]:
import os
import pandas as pd
import requests
from datetime import datetime
import json
import collections

In [2]:
%%time

# 读取2000年至2013年的每一天的 S&P 股票的资料,如果有股市开的话

# url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&outputsize=full&apikey=demo'
# web_data = requests.get(url)

API_KEY = "25IEBEKH83S6MQOE"
page_num = 1
page_size = 100
url_params = {"function":"TIME_SERIES_DAILY_ADJUSTED", "symbol": ".INX", "outputsize": "full", "apikey": API_KEY}
everything_url="https://www.alphavantage.co/query?"
web_data = requests.get(everything_url, params=url_params)

web_data.encoding = 'utf-8'
web_data_text = web_data.text
json_acceptable_string = web_data_text.replace("'", "\"")
web_data_dict = json.loads(json_acceptable_string)

print(type(web_data_dict), len(web_data_dict), web_data_dict.keys())

del web_data, web_data_text, json_acceptable_string

meta_data_dict = web_data_dict["Meta Data"]
stock_data_dict = web_data_dict["Time Series (Daily)"]

del web_data_dict

print(meta_data_dict, "\n")
print(len(stock_data_dict))

# print(type(stock_data_dict), stock_data_dict.keys())

# stock_time = list(stock_data_dict.keys())
# print(type(stock_time), stock_time[:3])

# stock_val = stock_data_dict.values()
# term = stock_data_dict["2018-05-24"]
# print(type(term), term, term["4. close"])

<class 'dict'> 2 dict_keys(['Meta Data', 'Time Series (Daily)'])
{'1. Information': 'Daily Time Series with Splits and Dividend Events', '4. Output Size': 'Full size', '3. Last Refreshed': '2018-06-14', '5. Time Zone': 'US/Eastern', '2. Symbol': 'S&P500 Index'} 

4642
CPU times: user 49.7 ms, sys: 49.2 ms, total: 99 ms
Wall time: 5.91 s


In [3]:
# 把获取到的股票data按日期排序,并按照今天比昨天是否增长设置1和0
# 今天比昨天价格更低为0,否则为1(可能是涨,可能是维持不变)
ostock_data_dict = collections.OrderedDict(sorted(stock_data_dict.items()))

new_stocks = dict()
new_stock_keys = list(ostock_data_dict.keys())
print(type(new_stock_keys))
old_key = new_stock_keys[0]
for now_key in new_stock_keys:
    if ostock_data_dict[now_key]["4. close"] < ostock_data_dict[old_key]["4. close"]:
        new_stocks[now_key] = 0
    else:
        new_stocks[now_key] = 1
    old_key = now_key
    
# Print data for checking
print(stock_data_dict["2000-01-03"], stock_data_dict["2000-01-03"]["4. close"])
print(len(new_stocks), len(ostock_data_dict))
print(new_stocks["2000-01-03"], stock_data_dict["2000-01-03"]["4. close"]) # first day from API
print(new_stocks["2000-01-04"], stock_data_dict["2000-01-04"]["4. close"])
print(new_stocks["2000-01-05"], stock_data_dict["2000-01-05"]["4. close"])
print(new_stocks["2006-10-20"], stock_data_dict["2006-10-20"]["4. close"]) # first day of news dataset
print(new_stocks["2010-12-31"], stock_data_dict["2010-12-31"]["4. close"]) # last day of news dataset

<class 'list'>
{'4. close': '1455.2200', '2. high': '1478.0000', '5. adjusted close': '1455.2200', '1. open': '1469.2500', '6. volume': '931800000', '8. split coefficient': '1.0000', '3. low': '1438.3600', '7. dividend amount': '0.0000'} 1455.2200
4642 4642
1 1455.2200
0 1399.4200
1 1402.1100
1 1368.6000
0 1257.6400


In [4]:
# Make sure news dataset has been extracted (follow instructions on README.md)
TRAINING_PATH = '../data/news/'
TESTING_PATH = '../data/news/'

In [5]:
old_categories = [dirname for dirname in os.listdir(TRAINING_PATH)]
categories= sorted(old_categories)
print("Number of categories: %d; first=%s, last=%s" % (len(categories), categories[0], categories[-1]))

Number of categories: 895; first=2006-10-20, last=2010-12-31


In [6]:
%%time

# Read News Headlines
train_list = []

for category in categories:
    category_path = TRAINING_PATH + category + "/"
    
    for filename in os.listdir(category_path):
        filepath = category_path + filename
        # print(filepath)
        with open(filepath, encoding='utf-8') as file:
            try:
                words = file.read().strip().split('-- http://www.bloomberg.com')
                if words[0] == "":
                    pass
                else:
                    abs_words = words[0].split("\n")
                    if len(abs_words) > 2:
                        # Remove '-- ' prefix of time and title
                        ari_title = abs_words[0].replace('-- ', '')
                        ari_time = abs_words[2].replace('-- ', '')
                        if category in stock_data_dict.keys():
                            price = new_stocks[category]
                            train_list.append([ari_time, ari_title, price])
            except ValueError:
                print (ValueError)
                break


CPU times: user 2.6 s, sys: 557 ms, total: 3.15 s
Wall time: 3.16 s


In [18]:
train_df = pd.DataFrame(train_list, columns=["ari_time", "ari_title", "price"])
print("Shape of training data frames:", train_df.shape)

# Print data for checking
for i in range(5):
    print(train_list[i][0], ':::', train_list[i][1])
    print(list(ostock_data_dict.keys())[i])

Shape of training data frames: (55483, 3)
2006-10-20T20:16:16Z ::: Inco's Net Soars on Higher Metal Prices, Breakup Fee
2000-01-03
2006-10-23T11:51:36Z ::: EU Energy Chief Backs German Plan for Price Controls
2000-01-04
2006-10-23T20:00:29Z ::: Ex-Plant Worker Shuster Pleads Guilty in Trading Case
2000-01-05
2006-10-24T10:53:59Z ::: Russia, Ukraine End Dispute That Cut Gas Supplies
2000-01-06
2006-10-24T01:32:04Z ::: Jim Cramer: Bare Escentuals, Allergan, Medicis, Avon
2000-01-07


In [8]:
%%time
train_df.to_pickle('train.pkl')

CPU times: user 28.4 ms, sys: 13.3 ms, total: 41.6 ms
Wall time: 39.1 ms


In [9]:
%time

# Read saved pickle file for checking

pickle_df = pd.read_pickle('train.pkl')
# train_df.equals(pickle_df)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs


In [21]:
print(pickle_df.shape)
print(type(pickle_df))
print(pickle_df[:30]["price"])

(55483, 3)
<class 'pandas.core.frame.DataFrame'>
0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     0
10    0
11    0
12    0
13    1
14    1
15    1
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    1
25    1
26    1
27    1
28    1
29    1
Name: price, dtype: int64
